In [1]:
import os
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
import seaborn as sns
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Dataset

C:\Users\USER\anaconda3\envs\preparation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = os.getcwd()
save_path = os.path.join(path,"save")

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(1016) # Seed 고정

In [4]:
# 데이터 로딩 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로.
            transform (callable, optional): 샘플에 적용될 Optional transform.
        """
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['img_path'].iloc[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 이미지 전처리 및 임베딩
transform_train = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.RandomGrayscale(p=0.25),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
transform_test = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])
train_data = CustomDataset(csv_file=os.path.join(path,"train.csv"), transform=transform_train)
train_loader = DataLoader(train_data, batch_size=8, shuffle=False)

In [6]:
# CNN 기반 Autoencoder 모델 정의
class AutoencoderCNN(nn.Module):
    def __init__(self):
        super(AutoencoderCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.15),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ELU(0.15),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.15),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ELU(0.15),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ELU(0.15),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(0.15),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ELU(0.15),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
model = AutoencoderCNN().to(device)
print(model)
# 손실 함수 및 최적화 기준 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 모델 학습
num_epochs = 2000
for epoch in range(num_epochs):
    running_loss = 0.0
    for data in train_loader:
        images = data
        images = images.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader.dataset)}")

AutoencoderCNN(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.15)
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ELU(alpha=0.15)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): LeakyReLU(negative_slope=0.15)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): ELU(alpha=0.15)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ELU(alpha=0.15)
    (2): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.15)
    (4): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (5): ELU(alpha=0.15)
    (6): ConvTranspose2d(16, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), outpu

Epoch [167/2000], Loss: 0.09741005444890456
Epoch [168/2000], Loss: 0.09707020677870988
Epoch [169/2000], Loss: 0.09718841790033618
Epoch [170/2000], Loss: 0.09733865763379934
Epoch [171/2000], Loss: 0.09693750812554024
Epoch [172/2000], Loss: 0.09692855466139709
Epoch [173/2000], Loss: 0.09684365424611759
Epoch [174/2000], Loss: 0.09696231478116882
Epoch [175/2000], Loss: 0.09672976225754465
Epoch [176/2000], Loss: 0.09670452878508769
Epoch [177/2000], Loss: 0.09679889210233106
Epoch [178/2000], Loss: 0.09635624142599777
Epoch [179/2000], Loss: 0.09706598929536174
Epoch [180/2000], Loss: 0.09677397407276529
Epoch [181/2000], Loss: 0.09635290262145056
Epoch [182/2000], Loss: 0.09662903848808137
Epoch [183/2000], Loss: 0.09645627939225362
Epoch [184/2000], Loss: 0.09670004427013262
Epoch [185/2000], Loss: 0.09657087747199994
Epoch [186/2000], Loss: 0.09594063849096567
Epoch [187/2000], Loss: 0.09651745086264722
Epoch [188/2000], Loss: 0.09636742967954824
Epoch [189/2000], Loss: 0.096256

Epoch [354/2000], Loss: 0.09155786551640067
Epoch [355/2000], Loss: 0.0914742002884547
Epoch [356/2000], Loss: 0.09163981273840291
Epoch [357/2000], Loss: 0.09154427856066977
Epoch [358/2000], Loss: 0.0915721715406073
Epoch [359/2000], Loss: 0.09160528912650587
Epoch [360/2000], Loss: 0.09134819708379781
Epoch [361/2000], Loss: 0.09177089479047927
Epoch [362/2000], Loss: 0.09124131916974072
Epoch [363/2000], Loss: 0.09157726871715464
Epoch [364/2000], Loss: 0.0913580166342113
Epoch [365/2000], Loss: 0.0916394163885027
Epoch [366/2000], Loss: 0.09112557265120493
Epoch [367/2000], Loss: 0.09122988011495609
Epoch [368/2000], Loss: 0.09116918852491558
Epoch [369/2000], Loss: 0.09100993985021619
Epoch [370/2000], Loss: 0.09153249471540183
Epoch [371/2000], Loss: 0.09127599683026193
Epoch [372/2000], Loss: 0.09133641152454654
Epoch [373/2000], Loss: 0.09129368632752011
Epoch [374/2000], Loss: 0.0912529082505356
Epoch [375/2000], Loss: 0.09127000985151165
Epoch [376/2000], Loss: 0.09134648313

Epoch [541/2000], Loss: 0.08946916356053151
Epoch [542/2000], Loss: 0.08972541704564027
Epoch [543/2000], Loss: 0.08951305664481132
Epoch [544/2000], Loss: 0.08967233844486201
Epoch [545/2000], Loss: 0.08996994873867348
Epoch [546/2000], Loss: 0.08965406717269074
Epoch [547/2000], Loss: 0.08969391917398838
Epoch [548/2000], Loss: 0.08960117039406243
Epoch [549/2000], Loss: 0.0896704184337401
Epoch [550/2000], Loss: 0.08974071147856018
Epoch [551/2000], Loss: 0.08965173027884792
Epoch [552/2000], Loss: 0.08965603482555336
Epoch [553/2000], Loss: 0.08972379799599939
Epoch [554/2000], Loss: 0.08949267815256343
Epoch [555/2000], Loss: 0.08963195468898111
Epoch [556/2000], Loss: 0.08942379249793263
Epoch [557/2000], Loss: 0.08957222737197025
Epoch [558/2000], Loss: 0.0897824237813972
Epoch [559/2000], Loss: 0.08953076121812695
Epoch [560/2000], Loss: 0.08952084788833985
Epoch [561/2000], Loss: 0.08969399535879842
Epoch [562/2000], Loss: 0.08946652234719953
Epoch [563/2000], Loss: 0.08953468

Epoch [728/2000], Loss: 0.08874892681277415
Epoch [729/2000], Loss: 0.08857035941221345
Epoch [730/2000], Loss: 0.08867050728327791
Epoch [731/2000], Loss: 0.08848554850883887
Epoch [732/2000], Loss: 0.08866732434627596
Epoch [733/2000], Loss: 0.08887405312257193
Epoch [734/2000], Loss: 0.08850975467565474
Epoch [735/2000], Loss: 0.08855516271132259
Epoch [736/2000], Loss: 0.08851075193411867
Epoch [737/2000], Loss: 0.0885446621148799
Epoch [738/2000], Loss: 0.08863016392843265
Epoch [739/2000], Loss: 0.08849280317064742
Epoch [740/2000], Loss: 0.08876605615229674
Epoch [741/2000], Loss: 0.08856319251334724
Epoch [742/2000], Loss: 0.08844215122047165
Epoch [743/2000], Loss: 0.08855264591918864
Epoch [744/2000], Loss: 0.08853426502204277
Epoch [745/2000], Loss: 0.08856525370072871
Epoch [746/2000], Loss: 0.08854250753289657
Epoch [747/2000], Loss: 0.08860612922990826
Epoch [748/2000], Loss: 0.08865769368382127
Epoch [749/2000], Loss: 0.08859621306680178
Epoch [750/2000], Loss: 0.0886002

Epoch [915/2000], Loss: 0.08807599068807324
Epoch [916/2000], Loss: 0.0881255868594971
Epoch [917/2000], Loss: 0.08814322567181967
Epoch [918/2000], Loss: 0.08789559894464385
Epoch [919/2000], Loss: 0.08806683896153186
Epoch [920/2000], Loss: 0.08796799424248682
Epoch [921/2000], Loss: 0.08797962061116393
Epoch [922/2000], Loss: 0.08805180589358012
Epoch [923/2000], Loss: 0.0881583226818434
Epoch [924/2000], Loss: 0.08809212192962987
Epoch [925/2000], Loss: 0.08783590268920845
Epoch [926/2000], Loss: 0.0881001826882922
Epoch [927/2000], Loss: 0.08784503773064681
Epoch [928/2000], Loss: 0.08813163666098331
Epoch [929/2000], Loss: 0.0881120234098233
Epoch [930/2000], Loss: 0.08815342964420855
Epoch [931/2000], Loss: 0.08833822461081223
Epoch [932/2000], Loss: 0.08824815436707975
Epoch [933/2000], Loss: 0.08806705915592086
Epoch [934/2000], Loss: 0.08797142056512161
Epoch [935/2000], Loss: 0.08807227865491114
Epoch [936/2000], Loss: 0.08798038742631814
Epoch [937/2000], Loss: 0.0879554502

Epoch [1100/2000], Loss: 0.08744589564665942
Epoch [1101/2000], Loss: 0.08739130031055128
Epoch [1102/2000], Loss: 0.08754585211125898
Epoch [1103/2000], Loss: 0.08759807393024785
Epoch [1104/2000], Loss: 0.08775297730741366
Epoch [1105/2000], Loss: 0.08755003220178711


In [ ]:
train_loss = running_loss/len(train_loader.dataset)
print(train_loss)

In [ ]:
train_dataset2 = CustomDataset(csv_file=os.path.join(path,"train.csv"), transform=transform_test)
train_loader2 = DataLoader(train_dataset2, shuffle=False)

correct = 0
total = 0
test_pred = []
loss_list = []
with torch.no_grad():
    for data in train_loader2:
        images = data
        images = images.to(device)
        outputs = model(images)
        loss = criterion(outputs, images).to(device).item()
        loss_list.append(loss)
# print(loss_list)
plt.hist(loss_list)
plt.show()
print(np.mean(loss_list))
m1 = np.percentile(np.array(loss_list), 50, axis=0)
print(m1)
            

In [ ]:
test_dataset = CustomDataset(csv_file=os.path.join(path,"test.csv"), transform=transform_test)
test_loader = DataLoader(test_dataset, shuffle=False)

correct = 0
total = 0
test_pred = []
loss_list = []
with torch.no_grad():
    for data in test_loader:
        images = data
        images = images.to(device)
        outputs = model(images)
        loss = criterion(outputs, images).to(device).item()
        loss_list.append(loss)
        if loss<=m1 :#or loss>=m2:
            test_pred.append(0.)
        else:
            test_pred.append(1.)
plt.hist(loss_list)
plt.show()            

In [ ]:
answer = [0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,1,1,0,1,0,0,1,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,0,0,1,1]
cf_matrix = confusion_matrix(answer, test_pred)
group_names = ["TN", "FP (type  II error)", "FN (type I error)", "TP"]
group_counts = [value for value in cf_matrix.flatten()]
group_percentages = [f"{value:.1%}" for value in cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n({v3})" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)
labels
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

In [ ]:
submit = pd.read_csv(os.path.join(path,"sample_submission.csv"))
submit['label'] = test_pred
submit.head()

In [ ]:
submit.to_csv(os.path.join(save_path,"submit63.csv"), index=False)